In [2]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt

import os 

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# reading imbd review data from http://ai.stanford.edu/~amaas/data/sentiment/

# reads the .txt files under dir and returns an ndarray of the documents
def read_docs(dir: str) -> np.ndarray:
    try:
        documents = []
        for filepath in sorted(os.listdir(dir), key=lambda name : int(name.split('_')[0])):
            with open(f'{dir}/{filepath}', 'r', encoding='utf8') as txtfile:
                txt = txtfile.read()
                documents.append(txt)
        
            # print(f'current_file: {filepath:<20}')
        return np.array(documents, dtype=str)

    except FileNotFoundError as e:
        print(f'{e}: unable to read directory {dir}')

docs = read_docs('./aclImdb/train/neg')
print(docs.shape)
print(docs[30])
# vectorizer = CountVectorizer()
# X_TRAIN = vectorizer.fit_transform()

(12500,)


In [9]:
print('1_1.txt' > '10000_1.txt')

True
